In [ ]:
import pandas as pd

df = pd.read_csv('df_Req_original.csv')

# Pré-processamento

In [ ]:
# Text cleaning

import nltk
nltk.download('punkt', quiet=True)
from nltk.tokenize import word_tokenize
import re
import string

df_Req=pd.read_csv('df_Req_original.csv')
##df_Req=pd.read_csv('df123.csv', sep = ',', delimiter = ',')

df_Req.sort_values(by='File_ID',inplace=True)
df_Req.set_index('File_ID')

print(df_Req.shape)

df_Est=pd.read_csv('estiDeep.data', names=['Estimate'])
#df_Est=pd.read_csv('estiDeepPre.data', names=['Estimate'])
#df_Est=pd.read_csv('estiDeepProj.data', names=['Estimate'])

df_Est.insert(0, 'Est_ID', range(0, len(df_Est)))



# Merging requirements and efforts estimates dataframes
# df_Est contains storypoints software effort estimatives

df_Merge=pd.merge(df_Req, df_Est, how='inner', on=None, left_on='File_ID', right_on='Est_ID',
         left_index=False, right_index=False, sort=False,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

# Removing unused columns

df_Merge.drop(columns =['Filename','File_ID','Est_ID'], inplace=True)
df=df_Merge

# Visualizing merged dataframe

print(df_Merge.head(3))
print(df.shape)
print(df.dtypes)
print(df.isnull().sum())

# Cleaning method - removes tags like < div> ,< p> , < code>, < pre> ...etc which are present in requirements texts

def CleanText(Text):
    Text = re.sub(r'html',' ',Text)
    Text = re.sub(r'{html}',' ',Text)
    Text = re.sub(r'<div>',' ',Text)
    Text = re.sub(r'<p>',' ',Text)
    Text = re.sub(r'<pre>',' ',Text)
    Text = re.sub(r'<code>',' ',Text)
    Text = re.sub(r'html',' ',Text)
    Text = re.sub(r'< div>',' ',Text)
    Text = re.sub(r'<div>',' ',Text)
    Text = re.sub(r'< p>',' ',Text)
    Text = re.sub(r'<p>',' ',Text)
    Text = re.sub(r'< pre>',' ',Text)
    Text = re.sub(r'< code>',' ',Text)
    Text = re.sub(r'< code>',' ',Text)
    
    # Uses str lib to make an additional cleaning of punctuation and digits (we considered irrelevant characters)
    trans_punct = str.maketrans('', '', string.punctuation)
    trans_digit = str.maketrans('', '', string.digits)
    Text = Text.translate(trans_punct)
    Text = Text.translate(trans_digit)
    Text = Text.lower()
    
    # additional pre-processing (text strip, and tokenizing)
    Text = Text.strip() # remoção de espaços em branco
    word_tokens = word_tokenize(Text) # tokenização do texto
    Text = ' '.join(word_tokens)
    return Text

df['Cleaned']= df['Text'].apply(CleanText)
print(df.columns)

# Verificando os dados
print(df.head(3))

# df contains the added columns: "Est_ID" which contains estimate, "Cleaned" which contains the cleaned requirement.
df.to_csv('df_Req_Final.csv', sep= ',', index = None)

print(df['Cleaned'].shape)


Mounted at /content/gdrive/
(23313, 4)
(23313, 3)
Index(['Unnamed: 0', 'Text', 'Estimate', 'Cleaned'], dtype='object')
   Unnamed: 0                                               Text  Estimate  \
0       23009  add ca against object literals in function inv...         1   
1       23013  update branding for appcelerator plugin to app...         1   
2       23003  create new json schema for sdk team{html}<div>...         1   

                                             Cleaned  
0  add ca against object literals in function inv...  
1  update branding for appcelerator plugin to app...  
2  create new json schema for sdk team create jso...  
(23313,)


# getting repo names

In [ ]:
import glob, os
filenames = glob.glob('repos/*.csv')
print(len(filenames),end='/')
print(len(os.listdir('repos/')),end=' files in folder\n')

16/17 files in folder


In [ ]:
import pandas as pd

data = []
for filename in filenames:
  df_aux = pd.read_csv(filename)
  df_aux['Filename'] = os.path.basename(filename)
  # print(df_aux['Filename'])
  data.append(df_aux)
print(len(data), 'readed files',end='\n\n')
print(sum(x.shape[0] for x in data), 'requirements')

16 arquivos lidos

23313 requisitos


In [ ]:
df_Req = pd.concat(data,ignore_index=True)
print(df_Req.shape)
df_Req.dropna()
print(df_Req.shape)

(23313, 5)
(23313, 5)


In [ ]:
df_Req['title'] = df_Req['title'].dropna()
df_Req['description'] = df_Req['description'].dropna()
df_Req['Text']= df_Req['title']+df_Req['description']
df_Req['Text']

0        HDFS ItemWriterBase integration of core HDFS w...
1        HDFS Core writing helper classesSimple file wr...
2                                                      NaN
3        Tuple data structureThe tuple data structure s...
4        Syslog IngestionHave a syslog.xml config file ...
                               ...                        
23308    Aptana Studio 3 gets killed after scrolling or...
23309    Issue configuring Bitnami Django stack with Ap...
23310    Syntax Error incorrectly reportedThe following...
23311    Aptana Fails to launch on OS X El Capitan (10....
23312    InstallDuring execution of Aptana_Studio_3_Set...
Name: Text, Length: 23313, dtype: object

In [ ]:
def na_prof(x):
  if(type(x)==float):
    return ''
  return x
df_Req['description'] = df_Req['description'].apply(lambda x: na_prof(x))
df_Req['title'] = df_Req['title'].apply(lambda x: na_prof(x))

In [ ]:
df_Req['Text']=df_Req['title']+''+df_Req['description']
df_Req['Text']=df_Req['Text'].apply(lambda x: x.lower())
df_Req.head(3)
teste = df_Req['Text'].iloc[:].values